In [36]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go
import settrade_v2
from settrade_v2 import Investor
import time

In [37]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

In [38]:
investor = Investor(
                app_id="rirpANWTJCOHVgnn",                                 
                app_secret="P7iCFYB6JX0cdLXogMwU0J2yNsrpne/nzX2pFy6kiA8=", 
                broker_id="SANDBOX",
                app_code="SANDBOX",
                is_auto_queue = False)

deri = investor.Derivatives(account_no="wanneeba-D")            

account_info = deri.get_account_info()

account_info       

{'creditLine': 2000000000.0,
 'excessEquity': 2000000000.0,
 'cashBalance': 2000000000.0,
 'equity': 2000000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 2000000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [44]:
#Get Candlestick
market = investor.MarketData()
res = market.get_candlestick(
symbol="AOT",
interval="1d",
normalized=True,)

In [43]:
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

,lastSequence,time,open,high,low,close,volume,value
0,349695,1677603600,73.00,73.50,72.50,72.50,13912341,1.013501e+09
1,349695,1677690000,72.50,72.50,71.25,71.75,34605326,2.487860e+09
2,349695,1677776400,71.50,71.50,70.25,70.50,25367513,1.791836e+09
3,349695,1678122000,70.75,71.00,69.75,69.75,26164171,1.837748e+09
4,349695,1678208400,69.25,70.00,69.00,69.75,32814418,2.278454e+09
...,...,...,...,...,...,...,...,...
495,349695,1742403600,65.25,70.00,65.00,70.00,4405101,2.908413e+08
496,349695,1742490000,69.00,91.25,59.25,67.00,4222400,2.981925e+08
497,349695,1742749200,64.75,65.25,64.00,65.25,2254101,1.456885e+08
498,349695,1742835600,52.00,52.00,50.00,52.00,314400,1.621875e+07


In [59]:
session.set_keyspace('data_stock')

In [60]:
session.execute("""
    CREATE TABLE IF NOT EXISTS candlestick_data (
        symbol TEXT,
        time TIMESTAMP PRIMARY KEY,
        open_price FLOAT,
        high_price FLOAT,
        low_price FLOAT,
        close_price FLOAT,
        volume INT,
        value FLOAT
    )
""")
print("✅ Keyspace และ Table พร้อมใช้งาน!")

✅ Keyspace และ Table พร้อมใช้งาน!


In [61]:
# ✅ โหลดรายชื่อหุ้นทั้งหมดจากไฟล์ CSV
try:
    symbols_df = pd.read_excel("get_stcok_name/stocth_names.xlsx")  # ไฟล์ต้องมีคอลัมน์ 'symbol'
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ 'stocth_names.xlsx' ใช้รายชื่อหุ้นที่กำหนดแทน")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

# ✅ ฟังก์ชันที่ดึงข้อมูล Financial Data สำหรับหุ้นแต่ละตัว
def insert_financial_data(symbol):
        for i in range(len(res["time"])):
            timestamp = datetime.fromtimestamp(res["time"][i])  
            open_price = res["open"][i]
            high_price = res["high"][i]
            low_price = res["low"][i]
            close_price = res["close"][i]
            volume = res["volume"][i]
            value = res["value"][i]

            # 👉 แทรกข้อมูลลงในตาราง
            session.execute("""
                INSERT INTO candlestick_data (symbol, time, open_price, high_price, low_price, close_price, volume, value)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """, (symbol, timestamp, open_price, high_price, low_price, close_price, volume, value))

        print(f"✅ เพิ่มข้อมูล {len(res['time'])} รายการของหุ้น {symbol} สำเร็จ!")

# ✅ ดึงข้อมูล Financial Data สำหรับหุ้นทุกตัว
for symbol in symbols:
    insert_financial_data(symbol)
    time.sleep(2)  # ⏳ ควบคุม API Rate Limit

✅ เพิ่มข้อมูล 500 รายการของหุ้น 24CS สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น 2S สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น 3BBIF สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น A สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น A5 สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น AAI สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น AAV สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น ABM สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น ACAP สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น ACC สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น ACE สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น ACG สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น ADB สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น ADD สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น ADVANC สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น ADVICE สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น AE สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น AEONTS สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น AF สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น AFC สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น AGE สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการของหุ้น AH สำเร็จ!
✅ เพิ่มข้อมูล 500 รายการขอ